<h2>Taller 10 - Pick and place usando control cartesiano</h2>
<br>Autor: Claudio Morales D.
<br>Centro de Automatización y Robótica Inacap
<br>Santiago, Chile, otoño 2023


Desde CoppeliaSim, abrir la escena 'scara_pick_n_place.ttt'


<h3>1. Nos conectamos y enviamos al robot a una posición inicial deseada</h3>

In [9]:
# Cargamos las librerías necesarias
import coppeliasim_zmqremoteapi_client
import time
from cartesian import delta_q


In [10]:
# creamos un cliente de conexión a CoppeliaSim y obtenemos acceso a sim
client = coppeliasim_zmqremoteapi_client.RemoteAPIClient()
sim = client.getObject('sim')

In [11]:
# Obtenemos los manejadores para las articulaciones y el actuador final
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
suctionPad = sim.getObject('/base/suctionPad')

print(j1, j2, j3, suctionPad)

# Enviamos al robot la posición deseada
q1_val = 0 * 3.1416/180
q2_val = 0.2
q3_val = 0 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

19 22 26 31


1

<h3>2. Verificamos que el suctionPad funciona bien</h3>

In [15]:
def setEffector(val):
# acciona el efector final (suctionPad)
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
    objectHandle = sim.getObject('/base/suctionPad')
    scriptHandle = sim.getScript(sim.scripttype_childscript,objectHandle)
    res = sim.callScriptFunction('setEffector',scriptHandle,val)
    return res

In [16]:
# verificamos si el Pad se activa y desactiva
setEffector(0)

'suctionPad OFF'

In [17]:
# 1. Nos posicionamos sobre la caja
q1_val = 45 * 3.1416/180
q2_val = 0.2
q3_val = 45 * 3.1416/180

sim.setJointTargetPosition(j1, q1_val)
sim.setJointTargetPosition(j2, q2_val)
sim.setJointTargetPosition(j3, q3_val)

1

In [18]:
# 2, Bajamos a la altura de la caja
q2_val = 0.022
sim.setJointTargetPosition(j2, q2_val)

1

In [19]:
#activamos el actuador
setEffector(1)

'suctionPad ON'

In [20]:
# y subimos
q2_val = 0.2
sim.setJointTargetPosition(j2, q2_val)

1

In [21]:
#desactivamos el actuador
setEffector(0)

'suctionPad OFF'

In [22]:
# recogemos con pausas de tiempo
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(1)
sim.setJointTargetPosition(j2, 0.2)
time.sleep(3)
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(0)
sim.setJointTargetPosition(j2, 0.2)

1

<h3>3. Movemos con control cartesiano </h3>

In [23]:
# 1. posicionamos el robot
q1_inicial = 45 * 3.1416/180
q2_inicial = 0.2
q3_inicial = 45 * 3.1416/180

sim.setJointTargetPosition(j1, q1_inicial)
sim.setJointTargetPosition(j2, q2_inicial)
sim.setJointTargetPosition(j3, q3_inicial)

1

In [402]:
# leemos la posicón actual
q1_actual = sim.getJointPosition(j1)
q2_actual = sim.getJointPosition(j2)
q3_actual = sim.getJointPosition(j3)
q_actual = [q1_actual, q2_actual, q3_actual]
print(q_actual)

# definimos el vector de desplazamiento
dx = 0.01
dy = 0
delta_x = [dx, dy, 0]

# calculamos la nueva posición
q_nuevo = sympy.Matrix(q_actual) + delta_q(q_actual, delta_x)
print(q_nuevo)

# enviamos al robot las nuevas posiciones
sim.setJointTargetPosition(j1, float(q_nuevo[0]))
sim.setJointTargetPosition(j2, float(q_nuevo[1]))
sim.setJointTargetPosition(j3, float(q_nuevo[2]))

[0.2732293213777428, 0.19996331350565238, 1.3815028059263783]
Matrix([[0.277497424612153], [0.199963313505652], [1.42326368383769]])


1

In [30]:
# recogemos con pausas de tiempo
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(1)
sim.setJointTargetPosition(j2, 0.2)
time.sleep(3)
sim.setJointTargetPosition(j2, 0.022)
time.sleep(3)
setEffector(0)
sim.setJointTargetPosition(j2, 0.2)

1

In [ ]:
#include <Wire.h> // Si es necesario utilizar I2C
#include <Joystick.h> // Biblioteca del Joystick

Joystick joystick; // Crear una instancia del Joystick

void setup() {
  Serial.begin(9600); // Inicializar comunicación serial
  joystick.begin(); // Inicializar Joystick
}

void loop() {
  // Leer coordenadas x, y y estado del botón del Joystick
  int x = joystick.getX();
  int y = joystick.getY();
  int buttonState = joystick.getButton();

  // Enviar datos por el puerto serial
  Serial.print("X:");
  Serial.print(x);
  Serial.print(",Y:");
  Serial.print(y);
  Serial.print(",Button:");
  Serial.println(buttonState);

  delay(100); // Retardo para evitar lecturas demasiado frecuentes
}

In [ ]:
// Librerías necesarias
#include <Wire.h>

// Pines para el joystick analógico
int pinX = A0;
int pinY = A1;
int pinButton = 2;

void setup() {
  Serial.begin(9600); // Inicializar comunicación serial
  pinMode(pinButton, INPUT_PULLUP); // Configurar el pin del botón como entrada con resistencia pull-up
}

void loop() {
  // Leer valores analógicos del joystick y estado del botón
  int x = analogRead(pinX);
  int y = analogRead(pinY);
  int buttonState = digitalRead(pinButton);

  // Enviar datos por el puerto serial
  Serial.print("X:");
  Serial.print(x);
  Serial.print(",Y:");
  Serial.print(y);
  Serial.print(",Button:");
  Serial.println(buttonState);

  delay(100); // Retardo para evitar lecturas demasiado frecuentes
}


In [29]:
#// Librerías necesarias
#include <Wire.h>

#// Pines para el joystick analógico
int pinX = A0;
int pinY = A1;
int pinButton = A2;

#// Variables para almacenar los valores del joystick y el estado del botón
int joyX;
int joyY;
int buttonState;
int prevButtonState = HIGH;
bool suctionActivated = false;

void setup() {
  Serial.begin(9600); // Inicializar comunicación serial
  pinMode(pinButton, INPUT_PULLUP); // Configurar el pin del botón como entrada con resistencia pull-up
}

void loop() {
  #// Leer valores analógicos del joystick y estado del botón
  joyX = analogRead(pinX);
  joyY = analogRead(pinY);
  buttonState = digitalRead(pinButton);

  #// Enviar los valores por el puerto serial
  Serial.print("X:");
  Serial.print(joyX);
  Serial.print(",Y:");
  Serial.print(joyY);
  Serial.print(",Button:");
  Serial.println(buttonState);

  #// Verificar si se ha pulsado el botón
  if (buttonState != prevButtonState) {
    if (buttonState == LOW) {
      // El botón ha sido pulsado
      suctionActivated = !suctionActivated; // Cambiar el estado de la ventosa
      if (suctionActivated) {
        Serial.println("Suction Activated");
      } else {
        Serial.println("Suction Deactivated");
      }
    }
    prevButtonState = buttonState;
  }

  delay(100); // #Retardo para evitar lecturas demasiado frecuentes
}


SyntaxError: invalid syntax (1064174889.py, line 2)

In [39]:
# librerías requeridas
import serial  # librería pyserial, para comunicación con los puertos serial < pip install pyserial >
import time    # para el manejo de eventos de tiempo, nativa de Python
import pandas as pd  # para el manejo de archivos de datos < pip install pandas >
import matplotlib.pyplot as plt # para realizar gráficos < pip install matplotlib >

# Establecer la conexión con el puerto serial
  serialport = serial.Serial('/dev/ttyUSB0', 9600)

# Establecer la conexión con la escena de CoppeliaSim
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://127.0.0.1:5555")  # Reemplaza con la dirección correcta

# Función para enviar comandos a CoppeliaSim
def enviar_comando(comando):
    socket.send_string(comando)
    respuesta = socket.recv()
    return respuesta

# Control de las articulaciones y el actuador de la ventosa
def controlar_robot(articulaciones, ventosa_activada):
    # Código para controlar las articulaciones y el actuador en CoppeliaSim
    # Utiliza la función enviar_comando para enviar los comandos adecuados

    # Ejemplo:
    comando = f"SetJointPosition articulacion1 {articulaciones[0]}"
    enviar_comando(comando)

    comando = f"SetJointPosition articulacion2 {articulaciones[1]}"
    enviar_comando(comando)

    comando = f"SetJointPosition articulacion3 {articulaciones[2]}"
    enviar_comando(comando)

    comando = f"SetActuatorState ventosa {ventosa_activada}"
    enviar_comando(comando)

# Loop principal
while True:
    # Leer los datos desde el puerto serial
    datos_serial = ser.readline().decode().strip()

    # Procesar los datos
    coordenadas = datos_serial.split(",")
    x = int(coordenadas[0].split(":")[1])
    y = int(coordenadas[1].split(":")[1])
    boton = int(coordenadas[2].split(":")[1])

    # Realizar el procesamiento y control del robot

    # Procesamiento de coordenadas del joystick para controlar las articulaciones
    angulo1 = map(x, 0, 1023, 0, 180)  # Mapear el valor de x a un rango de ángulos para la articulación 1
    angulo2 = map(y, 0, 1023, 0, 180)  # Mapear el valor de y a un rango de ángulos para la articulación 2

    # Control del actuador de la ventosa
    ventosa_activada = bool(boton)

    # Control del brazo robótico en CoppeliaSim
    articulaciones = [angulo1, angulo2, 0]  # Agrega los ángulos para la articulación 3 si es necesario
    controlar_robot(articulaciones, ventosa_activada)

    time.sleep(0.1)  # Tasa de lectura de aproximadamente 100 ms



IndentationError: unexpected indent (2586890871.py, line 8)